<a href="https://colab.research.google.com/github/KevinTheRainmaker/ML_DL_Basics/blob/master/HonGong_ML_DL/10_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증

### **키워드:** 검증 세트, 교차 검증, 그리드 서치, 랜덤 서치

검증 세트가 필요한 이유를 이해하고 교차 검증에 대해 알아보자. 그리드 서치와 랜덤 서치 방식을 이용해 하이퍼 파라미터 튜닝을 진행해보자.

## 검증 세트

테스트 세트로 일반화된 모델 성능을 정확하게 예측하려면 테스트 세트는 가장 마지막에 한 번만 사용하는 것이 가장 바람직하다. 그렇다면 최적의 매개변수 값을 찾는 하이퍼 파라미터 튜닝을 위해서는 어떻게 해야할까? 이때 필요한 것이 바로 검증 세트(Validation set)이다.

훈련 세트로 학습된 모델을 검증 세트로 평가하고, 매개변수를 바꿔가며 하이퍼 파라미터 튜닝을 수행한다. 그 다음, 훈련 세트와 검증 세트를 합쳐 모델을 다시 학습시킨 후 최종적으로 테스트 세트로 성능을 평가하면 된다.
 
전체 데이터에서 훈련 세트, 검증 세트, 테스트 세트의 비율은 일반적으로 3:1:1로 둔다.

In [2]:
# packages
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import uniform, randint

## 데이터셋 준비

In [3]:
wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
# data split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

`X_train`과 `y_train`을 다시 `train_test_split()` 함수에 넣어 훈련 세트와 검증 세트로 나누겠다.

In [8]:
X_sub, X_val, y_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [9]:
print(X_sub.shape, X_val.shape)

(4157, 3) (1040, 3)


이제 `X_sub`와 `y_sub`로 모델을 학습시키고 `X_val`과 `y_val`로 평가해보자.

In [10]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_sub, y_sub)
print(dt.score(X_sub, y_sub))
print(dt.score(X_val, y_val))

0.9971133028626413
0.864423076923077


## 교차 검증

학습 세트로부터 검증 세트를 분리한 탓에 학습 세트의 양이 줄었고, 이것은 모델 학습에 악영향을 끼칠 우려가 있다. 하지만 그렇다고 검증 세트를 너무 적게 쓰거나 쓰지 않는다면 제대로된 평가가 이루어지지 않을 것이다.

이럴 때 사용하는 방법이 교차 검증(Cross Validation)으로, 전체 훈련 세트에서 검증 세트로 분리되는 샘플을 바꿔가며 사용하는 방식이다.

전체 훈련 세트를 K개의 sub set로 나누어 K번 검증 세트를 바꾸는 방식을 K-fold Cross Validation이라고 한다. 평균적으로 K의 값으로는 5 또는 10을 주로 채택한다.

여기서 주의할 점은, 전달되는 데이터셋은 sub set이 아닌 전체 세트를 전달해야 한다는 점이다.

In [11]:
scores = cross_validate(dt, X_train, y_train)
print(scores)

{'fit_time': array([0.00955343, 0.00703406, 0.00727081, 0.00712538, 0.0068996 ]), 'score_time': array([0.00083828, 0.00064993, 0.00065947, 0.00067711, 0.00066996]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


`fit_time`, `score_time`, `test_score`이라는 키를 가진 딕셔너리를 반환한다. 앞의 두 개의 키는 각각 학습과 평가에 걸린 시간을 나타내며 `test_score` 키의 값은 K번째 검증 세트에 대한 테스트 점수를 의미한다.

기본값은 5로 설정되어있으며, `cross_validate`의 매개변수 `cv`를 이용해 폴드 수를 바꿀 수 있다.

In [12]:
print(np.mean(scores['test_score']))

0.855300214703487


앞서 우리는 `train_test_split()`을 이용해 sub set과 val set을 나누어 학습한 후 `cross_validate`를 했기 때문에 별도로 훈련 세트를 섞어 폴드를 나눠줄 필요가 없었다.

만약 그렇게 하지 않고 바로 진행을 하려한다면 분할기(splitter)를 따로 지정해줘야 할 필요가 있다.

In [13]:
scores = cross_validate(dt, X_train, y_train, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


분류의 경우 각 세트마다 존재하는 클래스의 비율을 일정하게 해주기 위해서 `StratifiedKFold()`를 사용한다.

만약 훈련 세트를 섞은 후 10-Fold CV를 진행하려면 다음과 같이 코드를 작성하면 된다.

In [14]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, X_train, y_train, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼 파라미터 튜닝 (Hyper Parameter Tuning, HPO)

모델이 학습하는 파라미터인 모델 파라미터와 달리 사용자가 직접 입력해주는 매개변수를 하이퍼 파라미터(Hyper Parameter)라고 한다.

모델마다 적게는 1~2개, 많게는 5~6개에 달하는 하이퍼 파라미터 값을 제공하며, 이를 바꿔가며 실험을 수행하고 최적의 하이퍼 파라미터 값을 찾는 과정을 하이퍼 파라미터 튜닝이라고 한다.

과거에는 이를 사용자가 일일히 해주었으나, 현재는 사람의 개입 없이 하이퍼 파라미터 튜닝을 수행하는 기술인 AutoML이 각광받고 있다.

HPO를 수행할 때는 한 가지 주의 사항이 있다. 

A 매개변수에 대해 최적의 값을 찾았다고 해서 이를 고정시키고 B 매개변수에 대한 탐색을 하면 안된다는 점이다. 보통 매개변수 간에는 유기적인 관계가 존재하고, 따라서 B 매개변수가 달라지면 최적의 A 매개변수도 달라질 수 있기 때문이다.

이러한 작업을 n중 for문으로 수행하면 굉장히 복잡하고 시간도 많이 걸릴 것이다. 그래서 우리는 `GridSearchCV`라는 사이킷런 제공 클래스를 사용할 것이다.

In [26]:
# configure
params = {'min_impurity_decrease':[0.0001,0.0005,0.001,0.005]} # 노드 분할을 위한 불순도 감소 최소량

In [27]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) 
# n_jobs 매개변수로 병렬 실행에 사용할 CPU 코어 수를 지정할 수 있다. 
# -1은 가능한 모든 CPU를 사용한다

gs.fit(X_train, y_train)
print(gs.score(X_train, y_train))
print(gs.score(X_test, y_test))

0.9615162593804117
0.8653846153846154


`GridSearchCV`는 기본적으로 5-fold CV를 수행하는데, `params`에서 바꿔주는 `min_impurity_decrease`가 5개이므로 5 * 5 = 25개의 모델을 학습시킨다.

이렇게 학습된 25개의 모델 중 최적의 매개변수를 가지고 학습된 모델은 `best_estimator_`를 통해 확인할 수 있으며, 이때 사용된 최적의 매개변수는 `best_params_`를 통해 확인할 수 있다.

In [28]:
dt = gs.best_estimator_
print(gs.best_params_)

print(dt.score(X_train, y_train))
print(dt.score(X_test, y_test))

{'min_impurity_decrease': 0.0001}
0.9615162593804117
0.8653846153846154


모델의 실행 점수는 위 gs의 결과값과 동일함을 확인할 수 있다.

각 매개변수에서 수행한 CV의 평균 점수가 확인하고 싶다면, `cv_results_` 속성의 `mea_test_score` 키에 저장된 값을 확인하면 된다.

In [29]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86761605 0.86165044 0.85318557]


넘파이의 `argmax()`를 이용하면 가장 큰 값의 인덱스를 출력할 수 있다.

In [30]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


`GridSearchCV`에 사용할 매개변수의 종류를 늘려 더 복잡한 HPO를 수행해보자.

In [34]:
# configure
params = {
    'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
    'max_depth':range(5,20,1),
    'min_samples_split':range(2,100,10) # 노드를 나누기 위한 최소 샘플 수
    } 

In [35]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) 
gs.fit(X_train, y_train)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score'])) # 최상의 교차 검증 점수

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


### +) 랜덤서치

매개변수의 값을 정하기 어렵거나, 너무 많은 매개변수가 있어 그리드서치 방식이 시간이 너무 오래 걸릴 때는 랜덤서치(Random Search) 방식을 사용할 수 있다.

랜덤 서치에는 매개변수 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달한다.

Scipy stats 서브 패키지 내 `uniform`과 `randint` 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다. 이러한 방식을 '균등 분포에서 샘플링'한다고 하는데, `randint`는 정숫값을, `uniform`은 실숫값을 뽑는다는 차이가 있다.

In [38]:
rgen = randint(0,10) # 0~10
rgen.rvs(10) # 10개

array([8, 7, 1, 9, 9, 7, 4, 2, 0, 5])

In [39]:
ugen = uniform(0,1) # 0~10
ugen.rvs(10)

array([0.30223857, 0.95153179, 0.26803982, 0.14241315, 0.15072332,
       0.48764667, 0.725663  , 0.8344644 , 0.77196743, 0.62427398])

위 방식을 사용해서 랜덤 서치를 진행해보자

In [40]:
# configure
params = {
    'min_impurity_decrease':uniform(0.0001, 0.001),
    'max_depth':randint(20,50),
    'min_samples_split':randint(2,25),
    'min_samples_leaf':randint(1,25) # 리프 노드가 되기 위한 최소 샘플 개수
    } 

In [41]:
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
rs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe6b4ed5690>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe6b4ed5910>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe6b44c6710>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe6b44c6150>},
                   random_state=42)

위 `params`에 정의된 매개변수 범위에서 총 100번(`n_iter`) 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾는다.

In [42]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [43]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [44]:
dt = rs.best_estimator_

print(dt.score(X_train, y_train))
print(dt.score(X_test, y_test))

0.8928227823744468
0.86
